In [ ]:

from huggingface_hub import login
login("HUggingfaceAPI")


In [ ]:
!pip install -q transformers accelerate bitsandbytes fastapi uvicorn nest-asyncio pyngrok
import torch
torch.cuda.empty_cache()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 11.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from fastapi import FastAPI
from collections import defaultdict
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import torch


In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True
)

model.eval()

print("Mistral 7B loaded successfully!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Mistral 7B loaded successfully!


In [ ]:
SYSTEM_PROMPT = """
You are "Eliya's AI Agent," a professional assistant for Eliya Abbas Sayyed’s Software Development Portfolio. He has a 2 years of experiance

STRICT DOMAIN BOUNDARY:
- Focus ONLY on Software Engineering, Data Analysis, and Eliya’s technical projects.
- This portfolio refers to software development work, NOT financial investments.
- When you are asked Hi, hello, how are you. just greet back no need to stumble

If the user asks about finance, investing, stocks, trading, or financial portfolios:
Respond ONLY with:
"I assist with Eliya's software engineering work, not financial investments."

KNOWLEDGE BASE:
- Role: Python Developer & Data Analyst (4+ years).
- Core Stack:
  • Python
  • SQL
  • Machine Learning
  • LLMs
  • Prompt Engineering
- Key Project 1:
  • QR-based Digital Menu (React, Node.js, MongoDB)
- Key Project 2:
  • Wallet Risk Scoring (DeFi transaction analysis using Python)

GUIDELINES:
1. Keep responses under 100 words if required to answer something big.
2. Use bullet points when listing technologies.
3. Do NOT invent new skills or projects.
4. If unknown detail:
   "I don't have that specific detail, but you can reach out to Eliya directly."
"""


In [ ]:
FINANCE_KEYWORDS = [
    "stock", "invest", "trading", "crypto", "market",
    "portfolio return", "shares", "equity", "forex"
]

PROFANITY_LIST = ["damn", "hell", "shit"]

def is_finance_query(text):
    text_lower = text.lower()
    return any(word in text_lower for word in FINANCE_KEYWORDS)

def contains_profanity(text):
    text_lower = text.lower()
    return any(word in text_lower for word in PROFANITY_LIST)


In [ ]:
session_memory = defaultdict(list)


In [ ]:
def confidence_score(response):
    score = 1.0

    if "I don't have that specific detail" in response:
        score -= 0.4

    if len(response.split()) < 12:
        score -= 0.2

    if "financial" in response.lower():
        score -= 0.5

    return round(max(score, 0.0), 2)


In [ ]:
GREETING_KEYWORDS = ["hi", "hello", "hey", "good morning", "good evening"]

def is_greeting(text):
    text_lower = text.lower().strip()
    return text_lower in GREETING_KEYWORDS


In [ ]:

def generate_response(user_input, session_id="default"):

    # 🔒 Hard finance guard
    if is_finance_query(user_input):
        return {
            "reply": "I assist with Eliya's software engineering work, not financial investments.",
            "confidence": 1.0
        }

    # 🚫 Profanity guard
    if contains_profanity(user_input):
        return {
            "reply": "Please keep the conversation professional.",
            "confidence": 1.0
        }

    memory = session_memory[session_id]

    memory.append({"role": "user", "content": user_input})

    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + memory[-6:]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=180,
            temperature=0.5,
            top_p=0.9,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response[len(prompt):].strip()

    memory.append({"role": "assistant", "content": response})

    score = confidence_score(response)

    return {
        "reply": response,
        "confidence": score
    }


In [ ]:
app = FastAPI()

@app.post("/chat")
async def chat(data: dict):
    user_message = data.get("message")
    session_id = data.get("session_id", "default")

    if not user_message:
        return {"error": "Message is required"}

    result = generate_response(user_message, session_id)

    return {
        "success": True,
        "reply": result["reply"],
        "confidence": result["confidence"]
    }


In [ ]:
ngrok.set_auth_token("YOURNGROKAPI")

nest_asyncio.apply()

public_url = ngrok.connect(8000)
print("Public URL:", public_url)

await uvicorn.Server(
    uvicorn.Config(app, host="0.0.0.0", port=8000)
).serve()


Public URL: NgrokTunnel: "https://unsober-iola-semielastically.ngrok-free.dev" -> "http://localhost:8000"


INFO:     Started server process [637]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     2401:4900:8817:6335:1195:5e89:abb9:1ecf:0 - "POST /chat HTTP/1.1" 200 OK
